In [101]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json

# Import API key
from api_keys import ZOMATO_API_KEY

# Incorporated citipy to determine city based on latitude and longitude
# from citipy import citipy

# Output File (CSV)
# output_data_file = "output_data/cities.csv"



In [102]:
loc_df = pd.read_csv('city_listShort.csv')
# loc_df = pd.read_csv('city_list.csv')
loc_df.dropna(inplace=True)
loc_df.head()

,cities,lat,long,population2010
0,Carefree,33.8223,-111.9182,"3,363"
1,Cave Creek,33.8334,-111.9507,"5,015"
2,Sedona,34.8697,-111.7610,"10,031"
3,Show Low,34.2542,-110.0298,"10,660"
4,Paradise Valley,33.5312,-111.9426,"12,820"


In [103]:
#build URLS

#url header/key
headers = {"user-key": ZOMATO_API_KEY}



In [104]:
#convert lat & long to string to pass to api
cLoc_data = loc_df
cLoc_data['lat'] = cLoc_data['lat'].astype(str)
cLoc_data['long'] = cLoc_data['long'].astype(str)

In [105]:
urlLocation = "https://developers.zomato.com/api/v2.1/locations?query="


location_data = []

i = 0
while i < len(loc_df):
    city = loc_df.iloc[i,0]
    lat = loc_df.iloc[i,1]
    long = loc_df.iloc[i,2]
    population = loc_df.iloc[i,3]
    
    location_url = urlLocation + city +'&lat='+lat+'&lon='+long

#     try:
    myLocations = requests.get(location_url, headers=headers).json()
    #print(myLocations)

    loc_list = myLocations["location_suggestions"]
    #print(loc_list)

    index=0

    entity_type = loc_list[index]["entity_type"]
    entity_id = loc_list[index]["entity_id"]
    city_name = loc_list[index]["title"]
    latitude = loc_list[index]["latitude"]
    longitude = loc_list[index]["longitude"]
    city_id = loc_list[index]["city_id"]
    city_name = loc_list[index]["city_name"]
    country_id = loc_list[index]["country_id"]
    country_name = loc_list[index]["country_name"]

    location_data.append({"Entity_type": entity_type,
                        "Entity_ID": entity_id,
                        "City_Name": city_name,
                        "Latitude": latitude,
                        "Longitude": longitude,
                        "City_ID": city_id,
                        "City_Lng_Name": city_name,
                        "City_Name": city,
                        "Country_ID": country_id,
                        "Country_Name": country_name,
                        "Population": population})


#     except:        
#         #print("City not found. Skipping...")
#         pass
    
    i +=1
    
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

-----------------------------
Data Retrieval Complete      
-----------------------------


In [106]:
#location_data
location_data_pd = pd.DataFrame(location_data)
location_data_pd.head()

,City_ID,City_Lng_Name,City_Name,Country_ID,Country_Name,Entity_ID,Entity_type,Latitude,Longitude,Population
0,301,Phoenix,Carefree,216,United States,198307,landmark,33.823888,-111.908740,"3,363"
1,11194,Maricopa County,Cave Creek,216,United States,118651,subzone,33.828572,-111.986521,"5,015"
2,469,Flagstaff,Sedona,216,United States,132640,subzone,34.865826,-111.782490,"10,031"
3,4387,Show Low,Show Low,216,United States,4387,city,34.245028,-110.033715,"10,660"
4,301,Phoenix,Paradise Valley,216,United States,118663,subzone,33.529861,-111.965868,"12,820"


In [107]:
#convert to strings for api

cLocation_data = location_data_pd
cLocation_data['Latitude'] = cLocation_data['Latitude'].astype(str)
cLocation_data['Longitude'] = cLocation_data['Longitude'].astype(str)
cLocation_data['Entity_ID'] = cLocation_data['Entity_ID'].astype(str)
# cLocation_data

In [108]:
restaruantData = []

# start = 1
# startStr = str(start)
count = 3
# i=0
print(len(cLocation_data))
n = 0
while n < len(cLocation_data):    
    i=0
    start = 1
    startStr = str(start)
#     
    entity_id = cLocation_data.iloc[n, 5]
    entity_type = cLocation_data.iloc[n, 6]

    entity_id = str(entity_id)

    while i < count:
#         urlSearch = "https://developers.zomato.com/api/v2.1/search?entity_id=301&entity_type=city&start=" +startStr+ "&sort=rating&order=desc"
        urlSearch = "https://developers.zomato.com/api/v2.1/search?entity_id="+entity_id+"&entity_type="+entity_type+"&start=" +startStr+ "&sort=rating&order=desc"
#         print(urlSearch)
                
        mySearchDetails = requests.get(urlSearch, headers=headers).json()

        results = mySearchDetails["results_found"]
        res_start = mySearchDetails["results_start"]
        res_shown = mySearchDetails["results_shown"]
        restaurants = mySearchDetails["restaurants"]
#         print(results)
#         print(res_start)
#         print(res_shown)
        
        for restaurant in restaurants:
             for key in restaurant:
        #             print(restaurant[key])
                Res_ID = restaurant[key]['id']
                Res_Name = restaurant[key]['name']
                Address = restaurant[key]['location']['address']
                Locality = restaurant[key]['location']['locality']
                City = restaurant[key]['location']['city']
                CityID = restaurant[key]['location']['city_id']
                Lat = restaurant[key]['location']['latitude']
                Long = restaurant[key]['location']['longitude']
                ZipCode = restaurant[key]['location']['zipcode']
                Cuisines = restaurant[key]['cuisines']
                Timings = restaurant[key]['timings']
                Avg_Cost_2 = restaurant[key]['average_cost_for_two']
                PriceRange = restaurant[key]['price_range']
                Rating = restaurant[key]['user_rating']['aggregate_rating']
                RatingText = restaurant[key]['user_rating']['rating_text']
                RatingColor = restaurant[key]['user_rating']['rating_color']
                Votes = restaurant[key]['user_rating']['votes']
                ReviewsCount = restaurant[key]['all_reviews_count']

                restaruantData.append({"Res_ID": Res_ID,
                                        "Res_Name":Res_Name,
                                        "Address":Address,
                                        "Locality":Locality,
                                        "City":City,
                                        "CityID":CityID,
                                        "Lat":Lat,
                                        "Long":Long,
                                        "ZipCode":ZipCode,
                                        "Cuisines":Cuisines,
                                        "Timings":Timings,
                                        "Avg_Cost_2":Avg_Cost_2,
                                        "PriceRange":PriceRange,
                                        "Rating":Rating,
                                        "RatingText":RatingText,
                                        "RatingColor":RatingColor,
                                        "Votes":Votes,
                                        "ReviewsCount":ReviewsCount})

        i += 1
        start = int(res_start) + int(res_shown)
        startStr = str(start)
#         print(startStr)
        
        if start == int(results):
            i=count
            continue
    
    n +=1

27
1
1
1
21
41
61
21
41
61
21
41
61
21
22
21
41
61
21
41
61
21
41
61
21
41
57
21
41
61
21
41
61
21
41
54
21
41
61
21
41
61
21
41
61
21
41
61
21
41
61
21
41
61
21
41
61
1
21
41
61
11
21
41
61
21
41
61
1
1
1
21
41
61
21
41
61


In [109]:
restaruantData_df = pd.DataFrame(restaruantData)
restaruantData_df.to_csv('Restaurants2.csv')

In [ ]:
##############################################

In [29]:
#LOCATION_DETAILS
locationDetailData = []
cityRestaurantData = []
topCuisineData = []
urlLocationDetails = "https://developers.zomato.com/api/v2.1/location_details?" #entity_id=11190&entity_type=city

i = 0
while i < len(cLocation_data):
    city_id = cLocation_data.iloc[i,0]
    city_LngName = cLocation_data.iloc[i,1]
    city_name = cLocation_data.iloc[i,2]    
    country_ID = cLocation_data.iloc[i,3]
    country_Name  = cLocation_data.iloc[i,4]
    entity_id = cLocation_data.iloc[i, 5]
    entity_type = cLocation_data.iloc[i, 6]
    Latitude = cLocation_data.iloc[i,7]
    Longitude = cLocation_data.iloc[i,8]
    Population = cLocation_data.iloc[i,9]
    
#     print(entity_id)
#     print(entity_type)

    LocDetailurl = urlLocationDetails + "entity_id="+entity_id+"&entity_type="+entity_type
    myLocationDetails = requests.get(LocDetailurl, headers=headers).json()
    
    popularity = myLocationDetails["popularity"]
    nightlife_index = myLocationDetails["nightlife_index"]
    nearby_res = myLocationDetails["nearby_res"]
    
    for restaurant in nearby_res:
        cityRestaurantData.append({"EntityID":entity_id,
                                  "RestaurantID":restaurant})
        
    top_cuisines = myLocationDetails["top_cuisines"]
    for cuisine in top_cuisines:
        topCuisineData.append({"EntityID":entity_id,
                           "TopCuisines": cuisine})
        
    
    subzone = myLocationDetails["subzone"]
    subzone_id = myLocationDetails["subzone_id"]
    #city = myLocationDetails["city"]
    num_restaurant = myLocationDetails["num_restaurant"]
    
    locationDetailData.append({"CityID": city_id,
                                "City_LngName": city_LngName,
                                "City_Name": city_name,
                                "EntityID":entity_id,
                                "EntityType":entity_type,
                                "Popularity":popularity,
                                "Nightlife_Index":nightlife_index,
                                "Subzone":subzone,
                                "SubzoneID": subzone_id,
                                "num_restaurant":num_restaurant,
                                "CountryID": country_ID,
                                "Country_Name": country_Name,
                                "Latitude":Latitude,
                                "Longitude":Longitude,
                                "Population":Population})
    
#     cityRestaurantData.append({"EntityID":entity_id,
#                               "Restaurant_list": nearby_res})
    
#     topCuisineData.append({"EntityID":entity_id,
#                            "Cuisines": top_cuisines})

    i += 1
    

In [30]:
location_details_pd = pd.DataFrame(locationDetailData)
location_details_pd.head()

,CityID,City_LngName,City_Name,CountryID,Country_Name,EntityID,EntityType,Latitude,Longitude,Nightlife_Index,Popularity,Population,Subzone,SubzoneID,num_restaurant
0,11195,Pinal County,Apache Junction,216,United States,118660,subzone,33.407421,-111.537498,0.19,1.46,"35,840",Apache Junction,118660,57
1,11194,Maricopa County,Avondale,216,United States,118657,subzone,33.4622206961,-112.3122858059,0.00,1.21,"76,238",Avondale,118657,134
2,4324,"Benson, Arizona",Benson,216,United States,4324,city,31.9699506154,-110.3080628179,0.00,2.14,"5,105",Benson,128479,181
3,4323,"Bisbee, Arizona",Bisbee,216,United States,4323,city,31.4247349729,-109.9102204034,0.64,1.48,"5,575",Bisbee,128476,215
4,11194,Maricopa County,Buckeye,216,United States,118627,subzone,33.421078,-112.612583,0.00,0.00,"50,876",Buckeye,118627,54


In [31]:
cityRestaurants_pd = pd.DataFrame(cityRestaurantData)
cityRestaurants_pd.head()

,EntityID,RestaurantID
0,118660,17027650
1,118660,17029785
2,118660,17029466
3,118660,17033599
4,118660,17035576


In [32]:
topCuisines_pd = pd.DataFrame(topCuisineData)
topCuisines_pd.head()

,EntityID,TopCuisines
0,118660,Fast Food
1,118660,American
2,118660,Mexican
3,118660,Chinese
4,118660,Italian


In [33]:
topCuisines_pd['sequence']=topCuisines_pd.groupby('EntityID').cumcount()
topCuisines_pd.head()

,EntityID,TopCuisines,sequence
0,118660,Fast Food,0
1,118660,American,1
2,118660,Mexican,2
3,118660,Chinese,3
4,118660,Italian,4


In [34]:
#join topCuisines with location for city name
cuisinesCity = pd.merge(topCuisines_pd, location_details_pd, on='EntityID', how='right')
cuisinesCity.head()

,EntityID,TopCuisines,sequence,CityID,City_LngName,City_Name,CountryID,Country_Name,EntityType,Latitude,Longitude,Nightlife_Index,Popularity,Population,Subzone,SubzoneID,num_restaurant
0,118660,Fast Food,0.0,11195,Pinal County,Apache Junction,216,United States,subzone,33.407421,-111.537498,0.19,1.46,"35,840",Apache Junction,118660,57
1,118660,American,1.0,11195,Pinal County,Apache Junction,216,United States,subzone,33.407421,-111.537498,0.19,1.46,"35,840",Apache Junction,118660,57
2,118660,Mexican,2.0,11195,Pinal County,Apache Junction,216,United States,subzone,33.407421,-111.537498,0.19,1.46,"35,840",Apache Junction,118660,57
3,118660,Chinese,3.0,11195,Pinal County,Apache Junction,216,United States,subzone,33.407421,-111.537498,0.19,1.46,"35,840",Apache Junction,118660,57
4,118660,Italian,4.0,11195,Pinal County,Apache Junction,216,United States,subzone,33.407421,-111.537498,0.19,1.46,"35,840",Apache Junction,118660,57


In [35]:
#add top Cuisine to location details
topCuisine = cuisinesCity['sequence']==0

LocationDetailAll = cuisinesCity[topCuisine]
LocationDetailAll.head()


,EntityID,TopCuisines,sequence,CityID,City_LngName,City_Name,CountryID,Country_Name,EntityType,Latitude,Longitude,Nightlife_Index,Popularity,Population,Subzone,SubzoneID,num_restaurant
0,118660,Fast Food,0.0,11195,Pinal County,Apache Junction,216,United States,subzone,33.407421,-111.537498,0.19,1.46,"35,840",Apache Junction,118660,57
5,118657,American,0.0,11194,Maricopa County,Avondale,216,United States,subzone,33.4622206961,-112.3122858059,0.00,1.21,"76,238",Avondale,118657,134
10,4324,Mexican,0.0,4324,"Benson, Arizona",Benson,216,United States,city,31.9699506154,-110.3080628179,0.00,2.14,"5,105",Benson,128479,181
15,4323,Pizza,0.0,4323,"Bisbee, Arizona",Bisbee,216,United States,city,31.4247349729,-109.9102204034,0.64,1.48,"5,575",Bisbee,128476,215
17,118627,Asian,0.0,11194,Maricopa County,Buckeye,216,United States,subzone,33.421078,-112.612583,0.00,0.00,"50,876",Buckeye,118627,54


In [36]:
#RESTAURANTS
RestData = []
urlRestaurant = "https://developers.zomato.com/api/v2.1/restaurant?res_id="

for RestaurantID in cityRestaurants_pd.RestaurantID: 
#     print(RestaurantID)
    Restaurant_url = urlRestaurant + RestaurantID
    RestaurantData = requests.get(Restaurant_url, headers=headers).json()
    
#     for key in RestaurantData:
#         print(key)
    
    RestID = RestaurantData["id"]
    RestName = RestaurantData["name"]
    URL = RestaurantData["url"]
    Address = RestaurantData["location"]["address"]
    Locality = RestaurantData["location"]["locality"]
    City = RestaurantData["location"]["city"]
    cityID = RestaurantData["location"]["city_id"]
    Lat = RestaurantData["location"]["latitude"]
    Long = RestaurantData["location"]["longitude"]
    ZipCode = RestaurantData["location"]["zipcode"]
    cuisine = RestaurantData["cuisines"]
    timings = RestaurantData["timings"]
    avgCost2 = RestaurantData["average_cost_for_two"]
    PriceRange = RestaurantData["price_range"]
    RatingAvg = RestaurantData["user_rating"]["aggregate_rating"]
    RatingTxt = RestaurantData["user_rating"]["rating_text"]
    RatingColor = RestaurantData["user_rating"]["rating_color"]
    
       
    RestData.append({"RestID":RestID,
                        "RestName":RestName,
                        "URL":URL,
                        "Address":Address,
                        "Locality":Locality,
                        "City":City,
                        "cityID":cityID,
                        "Lat": Lat,
                        "Long":Long,
                        "ZipCode":ZipCode,
                        "Cuisines":cuisine,
                        "Timings":timings,
                        "AvgCost2":avgCost2,
                        "PriceRange":PriceRange,
                        "RatingAvg":RatingAvg,
                        "RatingTxt":RatingTxt,
                        "RatingColor":RatingColor})
    

In [37]:
RestData_pd = pd.DataFrame(RestData)
RestData_pd.head()

,Address,AvgCost2,City,Cuisines,Lat,Locality,Long,PriceRange,RatingAvg,RatingColor,RatingTxt,RestID,RestName,Timings,URL,ZipCode,cityID
0,280 South Phelps Drive 85220,25,Pinal County,Mexican,33.4123700000,Apache Junction,-111.5505900000,2,3.6,9ACD32,Good,17027650,Los Gringos Locos,"11 AM to 8 PM (Mon-Thu, Sun), 11 AM to 9 PM (F...",https://www.zomato.com/pinal-county-az/los-gri...,85220,11195
1,"1422 E Broadway Ave, Apache Junction 85219",10,Pinal County,"American, Italian, Mexican",33.4080070000,Apache Junction,-111.5310220000,1,3.3,CDD614,Average,17029785,Tres Banderas,11 AM to 9 PM (Mon-Sun),https://www.zomato.com/pinal-county-az/tres-ba...,85219,11195
2,"945 E Scenic St, Apache Junction 85219",10,Pinal County,American,33.4185990000,Apache Junction,-111.5357360000,1,3.4,CDD614,Average,17029466,Superstition Skies,10 AM to 2 AM (Mon-Sun),https://www.zomato.com/pinal-county-az/superst...,85219,11195
3,"183 West Apache Trail, #4, Apache Junction 85120",10,Pinal County,Chinese,33.4125889000,Apache Junction,-111.5499210000,1,3.4,CDD614,Average,17033599,Lotus Garden Chinese,11 AM to 9 PM (Mon-Sun),https://www.zomato.com/pinal-county-az/lotus-g...,85120,11195
4,"285 N Apache Trail, Apache Junction 85120",70,Pinal County,Steak,33.4175130000,Apache Junction,-111.5469850000,4,3.2,CDD614,Average,17035576,Cowboy Up,"Lunch, Dinner (Sun, Mon, Tue, Wed, Thu, Fri, Sat)",https://www.zomato.com/pinal-county-az/cowboy-...,85120,11195


In [38]:
RestData_pd.to_csv('Restaurants.csv')
LocationDetailAll.to_csv('Location_Details.csv')